# A Markdown cell
- You can do all the fun stuff markdown has to offer

## Another Markdown cell

In [1]:
# This is a code cell
import pandas as pd
import plotly.express as pd

class A():
    def one():
        return 1

    def two():
        return 2

In [2]:
# This is another code cell
import numpy as np

mylist = np.arange(0,10)
print(f"this is my list {mylist}")

this is my list [0 1 2 3 4 5 6 7 8 9]
